# <font color='red'><b>Bootstrap assignment</b> </font>

<b>There will be some functions that start with the word "grader" ex: grader_sampples(), grader_30().. etc, you should not change those function definition.

Every Grader function has to return True.</b>

<font color='blue'> <b>Importing packages</b> </font>

In [1]:
import numpy as np # importing numpy for numerical computation
from sklearn.datasets import load_boston # here we are using sklearn's boston dataset
from sklearn.metrics import mean_squared_error # importing mean_squared_error metric
from sklearn.tree import DecisionTreeRegressor
from prettytable import PrettyTable

In [2]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable

In [3]:
print(x.shape,y.shape)

(506, 13) (506,)


## <font color='red'><b>Task 1</b></font>

<font color='red'> <b>Step - 1</b></font>

*  <font color='blue'><b>Creating samples</b></font><br>
    <b> Randomly create 30 samples from the whole boston data points</b>
    *  Creating each sample: Consider any random 303(60% of 506) data points from whole data set and then replicate any 203 points from the sampled points
    
     For better understanding of this procedure lets check this examples, assume we have 10 data points [1,2,3,4,5,6,7,8,9,10], first we take 6 data points randomly , consider we have selected [4, 5, 7, 8, 9, 3] now we will replicate 4 points from [4, 5, 7, 8, 9, 3], consder they are [5, 8, 3,7] so our final sample will be [4, 5, 7, 8, 9, 3, 5, 8, 3,7]
* <font color='blue'><b> Create 30 samples </b></font>
    *  Note that as a part of the Bagging when you are taking the random samples <b>make sure each of the sample will have different set of columns</b><br>
Ex: Assume we have 10 columns[1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,9 ,10] for the first sample we will select [3, 4, 5, 9, 1, 2] and for the second sample  [7, 9, 1, 4, 5, 6, 2] and so on...
Make sure each sample will have atleast 3 feautres/columns/attributes

In [4]:
def create_sample(data,y):
  indexes_303  = np.random.choice(data.shape[0],size = 303,replace=False)  # taking 303 indices randomly
  oob_indicies = [i for i in range(data.shape[0]) if i not in indexes_303] # taking out of bag points
  indexes_203  = np.random.choice(indexes_303.shape[0],size = 203)         # taking 203 indices from random 303 indices
  feature_indices = np.random.choice(data.shape[1],size =3,replace=False)  # taking any 3 feature indexes randomly

  sample       =  np.append(data[indexes_303,:],data[indexes_203,:],axis=0)
  sample       =  sample[:,feature_indices]
  
  target       =  np.append(y[indexes_303],y[indexes_203],axis=0)


  return sample,target,oob_indicies,feature_indices

<font color='red'><b>Step - 2 </b></font>

<font color='blue'><b>Building High Variance Models on each of the sample and finding train MSE value</b></font>

*  Build a regression trees on each of 30 samples.
*  Computed the predicted values of each data point(506 data points) in your corpus.
*  Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{30}\sum_{k=1}^{30}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$
*  Now calculate the $MSE =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$

In [5]:
y_pred  = np.zeros((30,506))
for_oob = np.zeros((30,506))
trees = []
features_per_tree = []

for i in range(30):

  sample,target,oob,fi = create_sample(x,y)

  features_per_tree.append(fi) 

  regressor = DecisionTreeRegressor(max_depth=None)
  regressor.fit(sample,target)

  y_pred[i,:] = regressor.predict(x[:,fi])

  oob_points = x[oob,:]
  oob_points = oob_points[:,fi]


  for_oob[i,oob] = regressor.predict(oob_points)

  trees.append(regressor)

*  <font color='blue'><b>Calculating the MSE score </b></font>

In [6]:
y_pred = (y_pred.sum(axis=0))/30
mse = (((y-y_pred)**2).sum(axis=0))/506

print("Mean squared error is: ",mse)

Mean squared error is:  3.8146387368896915


<font color='red'> <b>Step - 3 </b></font>

*  <font color='blue'><b>Calculating the OOB score </b></font>

*  Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{k}\sum_{\text{k= model which was built on samples not included } x^{i}}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.
*  Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.

In [7]:
k_s = np.count_nonzero(for_oob,axis=0)
oob_pred = (for_oob.sum(axis=0))/k_s
oob_score = (((oob_pred-y_pred)**2).sum(axis=0))/506
print("Out of bag score is: ",oob_score)

Out of bag score is:  5.692529569729782


# <font color='red'><b>Task 2</b></font>

*  <font color='blue'><b>Computing CI of OOB Score and Train MSE</b></font>
  *   Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score </li>
<li> After this we will have 35 Train MSE values and 35 OOB scores </li>
<li> using these 35 values (assume like a sample) find the confidence intravels of MSE and OOB Score </li>
<li> you need to report CI of MSE and CI of OOB Score </li>
<li> Note: Refer the Central_Limit_theorem.ipynb to check how to find the confidence intravel</li>
</ol>

In [8]:
def confidence_interval(features,targets):

  x = features
  y = targets

  mse_scores = []
  oob_scores = []

  for i in range(35):
    y_pred  = np.zeros((30,506))
    for_oob = np.zeros((30,506))
    trees = []
    features_per_tree = []

    for i in range(30):

      sample,target,oob,fi = create_sample(x,y)

      features_per_tree.append(fi) 

      regressor = DecisionTreeRegressor(max_depth=None)
      regressor.fit(sample,target)

      y_pred[i,:] = regressor.predict(x[:,fi])

      oob_points = x[oob,:]
      oob_points = oob_points[:,fi]


      for_oob[i,oob] = regressor.predict(oob_points)

      trees.append(regressor)
    
    y_pred = (y_pred.sum(axis=0))/30
    mse = (((y-y_pred)**2).sum(axis=0))/506

    k_s = np.count_nonzero(for_oob,axis=0)
    oob_pred = (for_oob.sum(axis=0))/k_s
    oob_score = (((oob_pred-y_pred)**2).sum(axis=0))/506

    mse_scores.append(mse)
    oob_scores.append(oob_score)
  
  return mse_scores,oob_scores




In [9]:
mse_scores,oob_scores = confidence_interval(x,y)

*  <font color='blue'><b>Computing CI (Background Theory)</b></font>
  *   First we will answer this question. What is meant by 95% confidence interval ? </li>
   - <b> 95% confidence interval does not eaxctly means that 95% of data lies in the gievn boundary. The exact meaning of that is if you are performing an experiment of taking samples from population and each sample have finite no.of observations and your keep on taking samples. Now 95% of times your population mean will fall in the confidence intervals calculated using the samples</b>.
<li> To calculate the confidence interval of population mean we have some methods. </li>
<li> Here we are assuming that we don't know the population standard deviation, 
now we will take the sample with finite no.of observations from the population and we will calculate the mean and standard deviation of the sample and we will assume the standard deviation of the population is same as the standard deviation of sample. So now the standard deviation of sample  = (standard deviation of sample)/sqrt(n). 
where n = no. of observations in the sample.  </li>
<li> 95% confidence interval = [sample_mean-2*standard deviation of sample ,  sample_mean+2*standard deviation of sample] </li>
</ol>

 - <font color='blue'><b>95% confidence interval for population mean of MSE</b></font>

In [18]:
mse_scores = np.asarray(mse_scores)
x = PrettyTable()
x = PrettyTable(["#samples", "Sample Size", "Sample mean", "Sample std", "Population mean","Left C.I","Right C.I","Catch"])

Population_mean = mse_scores.mean()

for i in range(10):

    sample = mse_scores[np.random.choice(mse_scores.shape[0],size = 20)]
    sample_size = len(sample)
    sample_mean = sample.mean()
    sample_std =  sample.std()

    # here we are using sample standard deviation instead of population standard deviation
    left_limit  = np.round(sample_mean - 2*(sample_std/np.sqrt(sample_size)), 3)
    right_limit = np.round(sample_mean + 2*(sample_std/np.sqrt(sample_size)), 3)

    row = []
    row.append(i+1)
    row.append(sample_size)
    row.append(sample_mean)
    row.append(sample_std)
    row.append(Population_mean)
    row.append(left_limit)
    row.append(right_limit)
    row.append((Population_mean <= right_limit) and (Population_mean >= left_limit))
    x.add_row(row)
print(x)

+----------+-------------+--------------------+--------------------+--------------------+----------+-----------+-------+
| #samples | Sample Size |    Sample mean     |     Sample std     |  Population mean   | Left C.I | Right C.I | Catch |
+----------+-------------+--------------------+--------------------+--------------------+----------+-----------+-------+
|    1     |      20     | 3.816616485836852  | 0.7996984170951091 | 4.1679015745287655 |  3.459   |   4.174   |  True |
|    2     |      20     | 4.049385947354119  | 0.774344078882585  | 4.1679015745287655 |  3.703   |   4.396   |  True |
|    3     |      20     | 3.6688417776465676 | 0.7697917714625971 | 4.1679015745287655 |  3.325   |   4.013   | False |
|    4     |      20     | 4.238493021343574  | 0.7805344934932933 | 4.1679015745287655 |  3.889   |   4.588   |  True |
|    5     |      20     | 4.457841497977688  |  0.93942477245271  | 4.1679015745287655 |  4.038   |   4.878   |  True |
|    6     |      20     | 4.171

 - <font color='blue'><b>95% Confidence interval for population mean of OOB Score</b></font>

In [19]:
oob_scores = np.asarray(oob_scores)
x = PrettyTable()
x = PrettyTable(["#samples", "Sample Size", "Sample mean", "Sample std", "Population mean","Left C.I","Right C.I","Catch"])

Population_mean = oob_scores.mean()

for i in range(10):

    sample = oob_scores[np.random.choice(oob_scores.shape[0],size = 20)]
    sample_size = len(sample)
    sample_mean = sample.mean()
    sample_std =  sample.std()

    # here we are using sample standard deviation instead of population standard deviation
    left_limit  = np.round(sample_mean - 2*(sample_std/np.sqrt(sample_size)), 3)
    right_limit = np.round(sample_mean + 2*(sample_std/np.sqrt(sample_size)), 3)

    row = []
    row.append(i+1)
    row.append(sample_size)
    row.append(sample_mean)
    row.append(sample_std)
    row.append(Population_mean)
    row.append(left_limit)
    row.append(right_limit)
    row.append((Population_mean <= right_limit) and (Population_mean >= left_limit))
    x.add_row(row)
print(x)

+----------+-------------+--------------------+--------------------+-------------------+----------+-----------+-------+
| #samples | Sample Size |    Sample mean     |     Sample std     |  Population mean  | Left C.I | Right C.I | Catch |
+----------+-------------+--------------------+--------------------+-------------------+----------+-----------+-------+
|    1     |      20     | 4.6841165191110665 | 0.8144850564367405 | 4.657297351316141 |   4.32   |   5.048   |  True |
|    2     |      20     | 4.660548067579326  | 0.6532932760543365 | 4.657297351316141 |  4.368   |   4.953   |  True |
|    3     |      20     | 4.550747820961018  | 0.7489167247742978 | 4.657297351316141 |  4.216   |   4.886   |  True |
|    4     |      20     | 4.793247739063346  | 0.5938873341917454 | 4.657297351316141 |  4.528   |   5.059   |  True |
|    5     |      20     | 4.537841375037493  | 0.8053855638969996 | 4.657297351316141 |  4.178   |   4.898   |  True |
|    6     |      20     | 4.78083001693

# <font color='red'><b>Task 3</b></font>

*  <font color='blue'><b>Given a single query point predict the price of house.</b></font>

Consider xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60] 
Predict the house price for this point as mentioned in the step 2 of Task 1.

In [10]:
xq= np.array([0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60])

predicted_value = 0

for model in range(30):
   predicted_value+=trees[i].predict(xq[features_per_tree[model]].reshape(1,-1))

print("The predicted output(House price) for the given query point is ",predicted_value/30)


The predicted output(House price) for the given query point is  [20.35666667]


<br><br><br>

<br><br><br>